In [1]:
# Dipakek
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import local_binary_pattern

# Fungsi untuk menghitung LBP pada satu gambar
def calculate_lbp_image(img):
    lbp_image = local_binary_pattern(img, 8, 1, method='uniform')
    return lbp_image

# Baca dataset FER2013
data = pd.read_csv('../fer2013.csv')

# Ekstraksi fitur LBP dan simpan dalam list
lbp_features = []
for i in range(len(data)):
    pixels = np.array(data['pixels'].iloc[i].split(' '), dtype=int)
    img = pixels.reshape(48, 48)
    lbp_image = calculate_lbp_image(img)
    lbp_features.append(lbp_image)

# Konversi list ke array
lbp_features = np.array(lbp_features)

# Konversi label emosi menjadi one-hot encoding
encoder = LabelEncoder()
labels = encoder.fit_transform(data['emotion'])
labels = tf.keras.utils.to_categorical(labels, num_classes=7)  # Ubah menjadi 7 karena ada 7 emosi dalam dataset

# Bagi data menjadi set pelatihan dan validasi
X_train, X_val, y_train, y_val = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)

# Buat model Sequential
model = Sequential()

# Layer 1
model.add(Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Output layer with 7 classes (emotions)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.save('lbpvgg.h5')

# Reshape data LBP untuk memenuhi input model
X_train = X_train.reshape(-1, 48, 48, 1)
X_val = X_val.reshape(-1, 48, 48, 1)

# Latih model dengan data LBP
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)


Epoch 1/10
449/449 [==============================] - 57s 80ms/step - loss: 2.7500 - accuracy: 0.2479 - val_loss: 1.7894 - val_accuracy: 0.2463
Epoch 2/10
449/449 [==============================] - 35s 78ms/step - loss: 1.7236 - accuracy: 0.3002 - val_loss: 1.6407 - val_accuracy: 0.3530
Epoch 3/10
449/449 [==============================] - 35s 78ms/step - loss: 1.4764 - accuracy: 0.4280 - val_loss: 1.5584 - val_accuracy: 0.3888
Epoch 4/10
449/449 [==============================] - 35s 78ms/step - loss: 1.0504 - accuracy: 0.6181 - val_loss: 1.6567 - val_accuracy: 0.3851
Epoch 5/10
449/449 [==============================] - 35s 78ms/step - loss: 0.4114 - accuracy: 0.8643 - val_loss: 1.9351 - val_accuracy: 0.3852
Epoch 6/10
449/449 [==============================] - 35s 78ms/step - loss: 0.1971 - accuracy: 0.9374 - val_loss: 2.5528 - val_accuracy: 0.4035
Epoch 7/10
449/449 [==============================] - 35s 78ms/step - loss: 0.1609 - accuracy: 0.9506 - val_loss: 2.4672 - val_accuracy:

In [3]:
model1 = model.save('lbpvgg10.h5')